In [1]:
#Imports

import numpy as np
import matplotlib.pyplot as plt
import hera_cal.abscal as abscal
import uvtools.dspec as dspec 
import hera_pspec
from hera_pspec import pspecbeam
from hera_pspec import pspecdata
import itertools
import scipy 
from scipy import signal
import pickle
import copy
from hera_cal.utils import polnum2str, polstr2num, jnum2str, jstr2num
from hera_cal.io import HERAData, HERACal
from hera_cal.io import DataContainer 
from hera_cal import apply_cal
from hera_cal import io
from hera_cal import smooth_cal
from hera_cal import vis_clean
from hera_cal import redcal
from pyuvdata import UVFlag
import glob
import tqdm
import os
import shutil
from hera_cal import frf
import imp
from hera_pspec import utils as uvp_utils
from hera_pspec import plot as pspec_plot
from hera_pspec import grouping

/users/kshahin/miniconda2/envs/hera3/lib/python3.7/site-packages/hera_sim/visibilities/__init__.py:22: UserWarning: HealVis failed to import.
  warnings.warn("HealVis failed to import.")
/users/kshahin/miniconda2/envs/hera3/lib/python3.7/site-packages/hera_sim/visibilities/__init__.py:27: UserWarning: PRISim failed to import.
  warnings.warn("PRISim failed to import.")
/users/kshahin/miniconda2/envs/hera3/lib/python3.7/site-packages/hera_sim/visibilities/__init__.py:33: UserWarning: VisGPU failed to import.
  warnings.warn("VisGPU failed to import.")
/users/kshahin/miniconda2/envs/hera3/lib/python3.7/site-packages/hera_sim/__init__.py:36: FutureWarning: 
In the next major release, all HERA-specific variables will be removed from the codebase. The following variables will need to be accessed through new class-like structures to be introduced in the next major release: 

noise.HERA_Tsky_mdl
noise.HERA_BEAM_POLY
sigchain.HERA_NRAO_BANDPASS
rfi.HERA_RFI_STATIONS

Additionally, the next maj

In [2]:
#Loading Data, Model, and Flags

flag_files = [f"H1C_Flags/{jd}.flags.h5" for jd in [2458098,2458099,2458101,2458102,2458103,2458104,2458105,2458106,
                                                    2458107,2458108,2458109,2458110,2458111,2458112,2458113,2458114,
                                                    2458115,2458116]]
model_files = sorted(glob.glob('/lustre/aoc/projects/hera/aewallwi/time_dep_flagging/2458098/*.true.uvh5'))
data_files = sorted(glob.glob('/lustre/aoc/projects/hera/aewallwi/time_dep_flagging/2458098/*.true.uvh5'))

In [17]:
#For-Loop Calibration

ntimes = []
for flags in flag_files:
    uvf = UVFlag(flags)
    ntimes.append(uvf.Ntimes)
nflagstimes = np.min(ntimes)
nfiles = []
for day,flag_file in enumerate(flag_files):
    if not os.path.exists(f'Day_{day}'):
        os.mkdir(f'Day_{day}')
    hdlst = io.HERAData(data_files)
    lsts = np.unique(np.hstack(list(hdlst.lsts.values())))
    dlst = np.mean(np.diff(lsts))
    flags = UVFlag(flag_file)
    #Selecting Frequency and LST Ranges
    flags.select(frequencies = flags.freq_array[(flags.freq_array>=115*1e+6) & (flags.freq_array<175*1e+6)])
    flags.select(times=flags.time_array[(flags.lst_array>=lsts.min()-dlst/2) & (flags.lst_array<=lsts.max()+dlst/2)])
    flags_chunks = [flags.flag_array.squeeze()[i*60:(i+1)*60] for i in range(nflagstimes//60)]
    file_number=0
    for data_file,model_file,flags_chunk in tqdm.tqdm(zip(data_files, model_files, flags_chunks)):
        #Opening Data & Flags
        hd_data = HERAData(data_file)     
        freqs = hd_data.freqs[(hd_data.freqs>=115*1e+6) & (hd_data.freqs<175*1e+6)]
        d_data, d_flags, d_nsamples = hd_data.read(polarizations=['nn'], frequencies=freqs)
        for bl in d_data:
            d_flags[bl] = flags_chunk
        hd_data.update(flags=d_flags)
        hd_data.write_uvh5(f"Day_{day}/data_{day}_{file_number}.uvh5", clobber=True)
        del d_data,d_flags,d_nsamples,hd_data
        #Calculate Gains
        redcal.redcal_run(input_data=f'Day_{day}/data_{day}_{file_number}.uvh5',
                          clobber=True, solar_horizon=90, verbose=True)
        abscal.post_redcal_abscal_run(data_file = f'Day_{day}/data_{day}_{file_number}.uvh5',
                                      redcal_file = f'Day_{day}/data_{day}_{file_number}.omni.calfits',
                                      model_files = [f'Day_{day}/data_{day}_{file_number}.uvh5'],
                                      clobber=True, data_solar_horizon=90,
                                      model_solar_horizon=90)
        file_number += 1
    nfiles.append(file_number)
    #Smooth Gains
    cs = smooth_cal.CalibrationSmoother(calfits_list=sorted(glob.glob(f'Day_{day}/data_{day}_*.abs.calfits')))
    cs.time_freq_2D_filter(time_scale=21600)
    cs.write_smoothed_cal(clobber=True,output_replace=('.abs.','.smooth_abs.'))
    
    for file_number in tqdm.tqdm(range(nfiles[-1])): 
        #Apply Smooth Gains
        apply_cal.apply_cal(data_infilename=f'Day_{day}/data_{day}_{file_number}.uvh5',
                            data_outfilename=f'Day_{day}/data_{day}_{file_number}_smoothcal.uvh5',
                            new_calibration=f'Day_{day}/data_{day}_{file_number}.smooth_abs.calfits', clobber=True)
        #Fourier-Filter
        vc = vis_clean.VisClean(f"DAY_{day}/data_{day}_{file_number}_smoothcal.uvh5")
        vc.read()
        vc.vis_clean(standoff=100, min_dly=600, mode='dpss_leastsq', skip_if_flag_within_edge_distance=1, 
                     flag_model_rms_outliers=True, max_contiguous_edge_flags=1)
        vc.write_filtered_data(filled_outfilename=f'Day_{day}/data_{day}_{file_number}_filtered.uvh5',clobber=True)
        os.remove(f'Day_{day}/data_{day}_{file_number}.uvh5')